In [1]:
# --- Podstawowe biblioteki ---
import pandas as pd
import numpy as np
from datetime import datetime
import os

# --- Biblioteki do Machine Learningu i śledzenia eksperymentów ---
from pycaret.regression import setup, compare_models, tune_model, finalize_model, save_model, predict_model, pull

In [2]:
# --- Wczytywanie Danych ---

# 1. Wczytanie głównego zbioru danych
try:
    print("Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...")
    df_original = pd.read_csv('Data_state_LSTM_predicted_full.csv', sep=';', low_memory=False)
    print(f"Wczytano pomyślnie. Kształt danych: {df_original.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać głównego pliku danych. {e}")
    df_original = pd.DataFrame()

# 2. Wczytanie słownika hierarchicznego
try:
    print("\nWczytuję słownik hierarchiczny: slownik_finalny_z_hierarchia.csv...")
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print(f"Słownik wczytany pomyślnie. Kształt danych: {df_slownik.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać pliku słownika. {e}")
    df_slownik = pd.DataFrame()

# 3. Wczytanie pliku z predykcjami lokalizacji z modelu LSTM
try:
    print("\nWczytuję predykcje lokalizacji: Saleflats_Loc.csv...")
    df_loc_predictions = pd.read_csv('Saleflats_Loc.csv', sep=';')
    # Wybieramy tylko kluczowe kolumny, aby uniknąć duplikatów
    df_loc_predictions = df_loc_predictions[['SaleId', 'Predict_Loc']]
    print(f"Predykcje lokalizacji wczytane pomyślnie. Kształt danych: {df_loc_predictions.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać pliku z predykcjami lokalizacji. {e}")
    df_loc_predictions = pd.DataFrame()

# --- Łączenie predykcji z głównym zbiorem danych ---
if not df_original.empty and not df_loc_predictions.empty:
    print("\nŁączę predykcje lokalizacji z głównym zbiorem danych...")
    # Używamy left merge, aby zachować wszystkie wiersze z df_original.
    # Wiersze spoza Mazowieckiego (nieobecne w df_loc_predictions) otrzymają NaN w kolumnie Predict_Loc.
    df_original = pd.merge(df_original, df_loc_predictions, on='SaleId', how='left')
    print(f"Połączenie zakończone. Nowy kształt danych: {df_original.shape}")
    print(f"Liczba wierszy z przypisaną predykcją lokalizacji: {df_original['Predict_Loc'].notna().sum()}")
else:
    # Jeśli plik predykcji nie został wczytany, tworzymy pustą kolumnę, aby reszta kodu działała
    if 'Predict_Loc' not in df_original.columns:
        df_original['Predict_Loc'] = np.nan

Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...
Wczytano pomyślnie. Kształt danych: (760765, 54)

Wczytuję słownik hierarchiczny: slownik_finalny_z_hierarchia.csv...
Słownik wczytany pomyślnie. Kształt danych: (747, 4)

Wczytuję predykcje lokalizacji: Saleflats_Loc.csv...
Predykcje lokalizacji wczytane pomyślnie. Kształt danych: (235700, 2)

Łączę predykcje lokalizacji z głównym zbiorem danych...
Połączenie zakończone. Nowy kształt danych: (760765, 55)
Liczba wierszy z przypisaną predykcją lokalizacji: 92076


In [3]:
# Funkcja do przygotowania danych, która będzie używana do treningu i finalnej predykcji
def prepare_and_unify_location_data(input_df, slownik_df):
    """
    Przygotowuje dane, włączając w to konwersję dat, łączenie ze słownikiem 
    i tworzenie jednej, zunifikowanej kolumny lokalizacyjnej na podstawie Predict_Loc.
    """
    df = input_df.copy()
    
    # 1. Inżynieria Cech (Inflacja)
    if 'AdjustedPrice' not in df.columns:
        print("Tworzenie kolumny 'AdjustedPrice'...")
        def adjust_price(row):
            price = pd.to_numeric(row['Price'], errors='coerce')
            if pd.isna(price): return np.nan
            date_str = row['NewestDate'] if 'NewestDate' in row and pd.notna(row['NewestDate']) else (row['DateAdded'] if 'DateAdded' in row and pd.notna(row['DateAdded']) else None)
            if date_str is None or date_str == 'NULL': return price
            try:
                offer_date = pd.to_datetime(date_str, errors='coerce')
                if pd.isna(offer_date): return price
                years_diff = (datetime.now() - offer_date).days / 365.25
                return round(price * (1.05**years_diff), 0) if years_diff > 0 else price
            except: return price
        df['AdjustedPrice'] = df.apply(adjust_price, axis=1)

    # 2. Konwersja typów
    for col in ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'StreetNumber']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    if 'BuiltYear' in df.columns:
        print("Konwertowanie 'BuiltYear' na format datetime...")
        median_year = pd.to_numeric(df['BuiltYear'], errors='coerce').median()
        df['BuiltYear'] = pd.to_numeric(df['BuiltYear'], errors='coerce').fillna(median_year)
        df.dropna(subset=['BuiltYear'], inplace=True)
        df['BuiltYear'] = df['BuiltYear'].astype(int).astype(str)
        df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')

    # 3. Integracja danych ze słownika (dla fallbacku)
    print("Integrowanie danych lokalizacyjnych ze słownika...")
    if 'StreetNumber' in df.columns and 'UlicaID' in slownik_df.columns:
        df.rename(columns={'StreetNumber': 'UlicaID'}, inplace=True, errors='ignore')
        df = pd.merge(df, slownik_df, on='UlicaID', how='left')
        for col in ['Dzielnica_Name', 'Ulica_Name']:
             if col in df.columns:
                df[col].fillna('Brak Danych', inplace=True)

    # 4. Tworzenie zunifikowanej kolumny lokalizacyjnej
    print("Tworzenie zunifikowanej kolumny 'Unified_Location'...")
    location_str_fallback = df['Dzielnica_Name'] + ' > ' + df['Ulica_Name']
    
    # Sprawdź, czy Predict_Loc istnieje (powinien po kroku 2)
    if 'Predict_Loc' not in df.columns:
        df['Predict_Loc'] = np.nan
    
    # GŁÓWNA LOGIKA: Użyj Predict_Loc, jeśli istnieje. Jeśli nie (dla innych województw), użyj fallbacku.
    df['Unified_Location'] = np.where(df['Predict_Loc'].notna(), df['Predict_Loc'], location_str_fallback)
    
    print("Przetwarzanie danych zakończone.")
    return df

# --- Główny przepływ ---
if not df_original.empty and not df_slownik.empty:
    df_processed = prepare_and_unify_location_data(df_original, df_slownik)
    
    data_for_training = df_processed.dropna(subset=['AdjustedPrice', 'Area']).copy()
    print(f"\nLiczba wierszy (przed czyszczeniem outlierów): {len(data_for_training)}")

    for col in ['AdjustedPrice', 'Area']:
        Q1 = data_for_training[col].quantile(0.25)
        Q3 = data_for_training[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        data_for_training = data_for_training[(data_for_training[col] >= lower_bound) & (data_for_training[col] <= upper_bound)]
    
    print(f"Liczba wierszy do treningu (po czyszczeniu): {len(data_for_training)}")
    print("Przykładowe dane z nową, zunifikowaną kolumną lokalizacyjną:")
    display(data_for_training[['Predict_Loc', 'Dzielnica_Name', 'Unified_Location']].head())
else:
    print("BŁĄD: Zbiór 'df_original' lub 'df_slownik' jest pusty. Przerwanie przetwarzania.")
    data_for_training = pd.DataFrame()

Tworzenie kolumny 'AdjustedPrice'...
Konwertowanie 'BuiltYear' na format datetime...
Integrowanie danych lokalizacyjnych ze słownika...
Tworzenie zunifikowanej kolumny 'Unified_Location'...
Przetwarzanie danych zakończone.

Liczba wierszy (przed czyszczeniem outlierów): 726257
Liczba wierszy do treningu (po czyszczeniu): 663110
Przykładowe dane z nową, zunifikowaną kolumną lokalizacyjną:


,Predict_Loc,Dzielnica_Name,Unified_Location
0,NaN,Brak Danych,Brak Danych > Brak Danych
1,NaN,Brak Danych,Brak Danych > Brak Danych
2,NaN,Brak Danych,Brak Danych > Brak Danych
3,NaN,Brak Danych,Brak Danych > Brak Danych
4,NaN,Brak Danych,Brak Danych > Brak Danych


In [4]:
if not data_for_training.empty:
    print("--- Konfiguracja eksperymentu PyCaret ---\n")
    
    numeric_features = ['Area', 'NumberOfRooms', 'Floor', 'Floors']
    date_features = ['BuiltYear']
    
    base_categorical_features = ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType']
    categorical_features = base_categorical_features + ['Unified_Location']

    print(f"Finalna lista cech kategorycznych do użycia w modelu: {categorical_features}\n")
    
    # Dodajemy Predict_Loc do ignorowanych, ponieważ jego informacja jest już w Unified_Location
    ignore_features = [
        'SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'BuildingCondition', 'Price',
        'DateAddedToDatabase', 'DateAdded', 'NewestDate', 'DateLastModification', 'DateLastRaises',
        'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice', 'PricePerSquareMeter',
        'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
        'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner',
        'Score', 'ScorePrecision', 'CommunityScore', 'NumberOfCommunityComments', 'NumberOfCommunityOpinions',
        'Archive', 'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
        'RegionNumber', 'SubRegionNumber', 'StreetNumber', 'EncryptedId', 'PredictedRenovation',
        'LocationPath', 'UlicaID', 'Predict_State', 'Predict_Loc', 
        'MiastoID', 'DzielnicaID', 'Wojewodztwo_Name', 'Miasto_Name', 'Dzielnica_Name', 'Ulica_Name'
    ]

    numeric_features_to_use = [c for c in numeric_features if c in data_for_training.columns]
    categorical_features_to_use = [c for c in categorical_features if c in data_for_training.columns]
    date_features_to_use = [c for c in date_features if c in data_for_training.columns]
    ignore_features_to_use = [c for c in ignore_features if c in data_for_training.columns]

    print(f"Cechy numeryczne: {numeric_features_to_use}")
    print(f"Cechy kategoryczne: {categorical_features_to_use}")
    print(f"Cechy daty: {date_features_to_use}")
    print(f"Liczba ignorowanych cech: {len(ignore_features_to_use)}")

    # Inicjalizacja środowiska PyCaret
    reg_exp = setup(
        data=data_for_training,
        target='AdjustedPrice',
        session_id=123,
        log_experiment=False, 
        numeric_features=numeric_features_to_use,
        categorical_features=categorical_features_to_use,
        date_features=date_features_to_use,
        ignore_features=ignore_features_to_use,
        normalize=True,
        normalize_method='zscore'
    )
else:
    print("BŁĄD: Zbiór 'data_for_training' jest pusty. Nie można uruchomić setup().")

--- Konfiguracja eksperymentu PyCaret ---

Finalna lista cech kategorycznych do użycia w modelu: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Unified_Location']

Cechy numeryczne: ['Area', 'NumberOfRooms', 'Floor', 'Floors']
Cechy kategoryczne: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Unified_Location']
Cechy daty: ['BuiltYear']
Liczba ignorowanych cech: 48


,Description,Value
0,Session id,123
1,Target,AdjustedPrice
2,Target type,Regression
3,Original data shape,"(663110, 60)"
4,Transformed data shape,"(663110, 17)"
5,Transformed train set shape,"(464176, 17)"
6,Transformed test set shape,"(198934, 17)"
7,Ignore features,48
8,Numeric features,4
9,Date features,1


In [5]:
# --- Bezpieczne Porównanie Modeli ---

if 'reg_exp' in locals() and reg_exp is not None:
    print("Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...")
    best_models_list = compare_models(sort='R2', n_select=3)

    if best_models_list:
        best_model = best_models_list[0]
        print("\n--- Najlepsze znalezione modele ---")
        print(best_models_list)
        print("\n--- Wybrany najlepszy model do dalszej pracy ---")
        print(best_model)
    else:
        print("\nBŁĄD: compare_models nie zwróciło żadnych modeli.")
        best_model = None
else:
    print("BŁĄD KRYTYCZNY: Eksperyment PyCaret (setup) nie został uruchomiony.")
    best_model = None

Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,112689.0653,25348735977.5585,159212.2131,0.6133,0.4818,42.5855,6.2230
catboost,CatBoost Regressor,126929.1740,28238012708.9707,168040.5698,0.5692,0.4908,41.6651,5.7950
xgboost,Extreme Gradient Boosting,127216.2375,28381730201.6000,168467.6125,0.5670,0.4907,41.9041,0.7610
lightgbm,Light Gradient Boosting Machine,129557.3751,29216895999.3760,170928.4478,0.5543,0.4960,42.5345,0.8260
dt,Decision Tree Regressor,132886.5442,40176670869.5128,200439.7656,0.3871,0.5874,40.4583,0.6180
dummy,Dummy Regressor,205441.9406,65550776729.6000,256026.7938,-0.0000,0.6294,53.0127,0.3700



--- Najlepsze znalezione modele ---
[RandomForestRegressor(n_jobs=-1, random_state=123), <catboost.core.CatBoostRegressor object at 0x000001D7F5DF2F50>, XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=-1, num_parallel_tree=None, ...)]

--- Wybrany najlepszy model do dalszej pracy ---
RandomForestRegressor(n_jobs=-1, random_st

In [6]:
# --- KROK: TUNE_MODEL ---

if 'best_model' in locals() and best_model is not None:
    print(f"Rozpoczynam tuning najlepszego modelu: {type(best_model).__name__}")
    tuned_best_model = tune_model(best_model, n_iter=20, optimize = 'R2')
    
    print("\n--- Wyniki po tuningu ---")
    tuned_results = pull()
    display(tuned_results)
    
    best_model = tuned_best_model 
else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Rozpoczynam tuning najlepszego modelu: RandomForestRegressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,132318.3028,29982692558.9431,173155.1113,0.5397,0.4947,22.0155
1,132868.5645,30098225548.2395,173488.4018,0.5373,0.5036,35.4258
2,133182.9865,30311581861.2207,174102.2167,0.5397,0.5244,56.9340
3,133116.1963,30335817065.9103,174171.8033,0.5374,0.5025,60.8967
4,132926.7804,30217683765.5894,173832.3438,0.5376,0.5084,18.6331
5,132830.9694,30010956776.2173,173236.7074,0.5447,0.5227,87.1849
6,132081.0012,29758807942.4353,172507.4142,0.5384,0.4824,33.6774
7,132708.9716,30058358276.5815,173373.4647,0.5437,0.5049,66.6316
8,132909.9894,30217986555.4074,173833.2148,0.5416,0.4975,26.1166


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).

--- Wyniki po tuningu ---


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,132318.3028,2.998269e+10,173155.1113,0.5397,0.4947,22.0155
1,132868.5645,3.009823e+10,173488.4018,0.5373,0.5036,35.4258
2,133182.9865,3.031158e+10,174102.2167,0.5397,0.5244,56.9340
3,133116.1963,3.033582e+10,174171.8033,0.5374,0.5025,60.8967
4,132926.7804,3.021768e+10,173832.3438,0.5376,0.5084,18.6331
5,132830.9694,3.001096e+10,173236.7074,0.5447,0.5227,87.1849
6,132081.0012,2.975881e+10,172507.4142,0.5384,0.4824,33.6774
7,132708.9716,3.005836e+10,173373.4647,0.5437,0.5049,66.6316
8,132909.9894,3.021799e+10,173833.2148,0.5416,0.4975,26.1166


In [7]:
# --- Finalizacja, Zapis i Predykcja ---

if 'best_model' in locals() and best_model is not None:
    print("Finalizowanie najlepszego modelu...")
    final_model = finalize_model(best_model)
    print("Sfinalizowany model:")
    print(final_model)

    model_filename = 'final_price_model'
    save_model(final_model, model_filename)
    print(f"\nModel został zapisany jako '{model_filename}.pkl'")

    print(f"\n\nWykonywanie predykcji na całym oryginalnym zbiorze danych...")
    
    # Używamy tej samej funkcji co w Komórce 3, aby zapewnić spójność
    print("\nPrzygotowuję oryginalny zbiór danych do predykcji...")
    data_for_prediction = prepare_and_unify_location_data(df_original, df_slownik)
    
    # Usuwamy kolumnę docelową przed predykcją
    if 'AdjustedPrice' in data_for_prediction.columns:
        data_for_prediction_clean = data_for_prediction.drop(columns=['AdjustedPrice'])
    else:
        data_for_prediction_clean = data_for_prediction

    # Przekazujemy do funkcji predict_model dane bez kolumny celu
    all_data_predictions = predict_model(final_model, data=data_for_prediction_clean)
    print("\nPredykcja zakończona.")

    # Zmieniamy nazwę kolumny z predykcją
    all_data_predictions.rename(columns={'prediction_label': 'PredictedPrice'}, inplace=True)
    
    # --- KLUCZOWA POPRAWKA: Poprawne budowanie finalnej ramki danych ---
    # Tworzymy finalny plik wyjściowy na bazie oryginalnych danych
    df_final_output = df_original.copy()
    
    # Dodajemy kolumnę AdjustedPrice z przetworzonych danych (zachowują ten sam indeks)
    df_final_output['AdjustedPrice'] = data_for_prediction['AdjustedPrice']
    
    # Dodajemy kolumnę z predykcją (również zachowuje ten sam indeks)
    df_final_output['PredictedPrice'] = all_data_predictions['PredictedPrice']
    
    output_filename = 'sale_2024_0_predict.csv'
    df_final_output.to_csv(output_filename, index=False, sep=';', decimal='.')

    print(f"\nWyniki zostały zapisane do pliku: {output_filename}")
    print("\nPrzykładowe dane z finalnego pliku:")
    # Ta linia teraz zadziała, ponieważ df_final_output zawiera wszystkie potrzebne kolumny
    display(df_final_output[['SaleId', 'Price', 'AdjustedPrice', 'PredictedPrice']].head(10))

else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Finalizowanie najlepszego modelu...
Sfinalizowany model:
Pipeline(memory=Memory(location=None),
         steps=[('date_feature_extractor',
                 TransformerWrapper(include=['BuiltYear'],
                                    transformer=ExtractDateTimeFeatures())),
                ('numerical_imputer',
                 TransformerWrapper(include=['Area', 'NumberOfRooms', 'Floor',
                                             'Floors'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['BuildingType', 'TypeOfMarke...
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['OwnerType', 'Unified_Location'],
                                    transformer=TargetEncoder(cols=['OwnerType'


Predykcja zakończona.

Wyniki zostały zapisane do pliku: sale_2024_0_predict.csv

Przykładowe dane z finalnego pliku:


,SaleId,Price,AdjustedPrice,PredictedPrice
0,88,766500.0,826479.0,852991.382667
1,99,540000.0,579926.0,438352.002863
2,115,540000.0,579539.0,547445.437000
3,140,544000.0,566773.0,580987.927500
4,145,459000.0,491097.0,381920.214150
5,159,779000.0,834700.0,799767.878167
6,165,359000.0,385904.0,405008.575000
7,173,380000.0,406627.0,428619.455833
8,189,354000.0,379109.0,375271.940000
9,208,820000.0,880276.0,839721.676667
